# **CUPED (Controlled-experiment Using Pre-Experiment Data)**

# Publication "Reducing variance in A/B testing with CUPED"

In [1]:
# Ноутбук-первоисточник https://github.com/mtrencseni/playground/blob/master/CUPED.ipynb

In [2]:
import scipy
import numpy as np
from math import sqrt
from scipy import stats
from numpy.random import normal
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def get_cuped_adjusted(a_before, b_before, a_after, b_after):
    cv = np.cov([a_after + b_after, a_before + b_before])
    theta = cv[0, 1] / cv[1, 1]
    mean_before = np.mean(a_before + b_before)
    a_after_adjusted = [after - (before - mean_before) * theta for after, before in zip(a_after, a_before)]
    b_after_adjusted = [after - (before - mean_before) * theta for after, before in zip(b_after, b_before)]
    return a_after_adjusted, b_after_adjusted

In [4]:
# def get_cuped_adjusted(a_before, b_before, a_after, b_after):
#     theta = np.cov([a_after + b_after, a_before + b_before], bias=True)[0][1] / np.var([a_before + b_before])
#     mean_before = np.mean(a_before + b_before)
#     a_after_adjusted = [after - (before - mean_before) * theta for after, before in zip(a_after, a_before)]
#     b_after_adjusted = [after - (before - mean_before) * theta for after, before in zip(b_after, b_before)]
#     return a_after_adjusted, b_after_adjusted

In [5]:
def get_ab_samples(before_mean, before_sigma, eps_sigma, treatment_lift, N):
    a_before = list(normal(loc=before_mean, scale=before_sigma, size=N))
    b_before = list(normal(loc=before_mean, scale=before_sigma, size=N))
    a_after  = [x + normal(loc=0, scale=eps_sigma) for x in a_before]
    b_after  = [x + normal(loc=0, scale=eps_sigma) + treatment_lift for x in b_before]
    return a_before, b_before, a_after, b_after

In [6]:
def lift(a, b):
    return np.mean(a) - np.mean(b)

In [7]:
def p_value(a, b):
    return stats.ttest_ind(a, b)[1]

In [8]:
N = 1000
before_mean = 100 
before_sigma = 50
eps_sigma = 20
treatment_lift = 2

In [9]:
a_before, b_before, a_after, b_after = get_ab_samples(before_mean, before_sigma, eps_sigma, treatment_lift, N)
a_after_adjusted, b_after_adjusted = get_cuped_adjusted(a_before, b_before, a_after, b_after)

In [10]:
print('A mean before = %05.1f, A mean after = %05.1f' % (np.mean(a_before), np.mean(a_after)))
print('B mean before = %05.1f, B mean after = %05.1f' % (np.mean(b_before), np.mean(b_after)))
print('Traditional    A/B test evaluation, lift = %.3f, p-value = %.3f' % (lift(a_after, b_after), p_value(a_after, b_after)))
print('CUPED adjusted A/B test evaluation, lift = %.3f, p-value = %.3f' % (lift(a_after_adjusted, b_after_adjusted),
                                                                           p_value(a_after_adjusted, b_after_adjusted)))

A mean before = 101.2, A mean after = 101.5
B mean before = 102.0, B mean after = 103.7
Traditional    A/B test evaluation, lift = -2.196, p-value = 0.349
CUPED adjusted A/B test evaluation, lift = -1.380, p-value = 0.120


# Вариант 2

In [11]:
print(f"Mean A before {np.round(np.mean(a_before),2)}")
print(f"Mean A after {np.round(np.mean(a_after),2)}")
print(f"Mean B before {np.round(np.mean(b_before),2)}")
print(f"Mean B after {np.round(np.mean(b_after),2)}")

Mean A before 101.17
Mean A after 101.54
Mean B before 101.99
Mean B after 103.73


In [12]:
_, pvalue_before = stats.ttest_ind(b_before, a_before)
_, pvalue_after = stats.ttest_ind(b_after, a_after)
print(f'pvalue_before {pvalue_before:0.3f}')
print(f'pvalue_after {pvalue_after:0.3f}')

pvalue_before 0.707
pvalue_after 0.349


In [13]:
def calculate_theta(a_after, b_after, a_before, b_before) -> float:
    """
    a_after - контрольная группа во время теста
    b_after - тестовая группа во время теста
    a_before - контрольная группа до теста
    b_before - тестовая группа до теста
    """
    y = np.hstack([a_after, b_after])
    y_cov = np.hstack([a_before, b_before])
    covariance = np.cov(y_cov, y)[0, 1]
    variance = y_cov.var()
    theta = covariance / variance
    return theta

In [14]:
def get_cuped_adjusted(a_before, b_before, a_after, b_after):
    theta = calculate_theta(a_after, b_after, a_before, b_before)
    a_after_adjusted = list(np.array(a_after) - theta * np.array(a_before))
    b_after_adjusted = list(np.array(b_after) - theta * np.array(b_before))
    return a_after_adjusted, b_after_adjusted

In [15]:
a_after_adjusted, b_after_adjusted = get_cuped_adjusted(a_before, b_before, a_after, b_after)

In [16]:
_, pvalue_cuped = stats.ttest_ind(a_after_adjusted, b_after_adjusted)
print(f'pvalue cuped {pvalue_cuped:0.3f}')

pvalue cuped 0.120


In [17]:
var_b_after = np.var(b_after)
var_a_after = np.var(a_after)
var_b_after_adjusted = np.var(b_after_adjusted)
var_a_after_adjusted = np.var(a_after_adjusted)

delta_y = np.mean(b_after) - np.mean(a_after)
delta_y_cuped = np.mean(b_after_adjusted) - np.mean(a_after_adjusted)

print(
    f'Тестовая группа\n    var(y) = {var_b_after:0.1f}\n    var(y_cuped) = {var_b_after_adjusted:0.1f}'
    f'\n    var(y)/var(y_cuped) = {var_b_after/var_b_after_adjusted:0.2f}'
)
print(
    f'Контрольная группа\n    var(y) = {var_a_after:0.1f}\n    var(y_cuped) = {var_a_after_adjusted:0.1f}'
    f'\n    var(y)/var(y_cuped) = {var_a_after/var_a_after_adjusted:0.2f}'
)
print(f'\ndelta_y = {delta_y:0.2f}\ndelta_y_cuped = {delta_y_cuped:0.2f}')

Тестовая группа
    var(y) = 2808.4
    var(y_cuped) = 386.5
    var(y)/var(y_cuped) = 7.27
Контрольная группа
    var(y) = 2693.5
    var(y_cuped) = 400.8
    var(y)/var(y_cuped) = 6.72

delta_y = 2.20
delta_y_cuped = 1.38
